In [2]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [1]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = 'xxx'

# the API call
query = 'factory+automation' # topic is "large language model"factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [2]:
import openai

openai.api_key = 'xxx'

#System prompt (for all users)
system_prompt = f""" Given Problem: Lack of real-time visibility and control over production processes
Pain Point: Manufacturing companies face inefficiency and delays in production due to a lack of real-time visibility and control over their processes, leading to increased costs and missed deadlines.
Most Affected Customer: Medium to large-sized manufacturing companies.
Reasoning: This problem affects a significant number of customers and can be addressed by providing real-time visibility and control, improving overall efficiency and responsiveness in production processes. 
     """

#Iteration1 prompt (Crazy Person)
iteration1_solutions = {}
i = 0

for doc in mergeflow_docs: #stores all the problems in 1 string 
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```    
  
    Imagine you are a crazy dreamer and entrepreneur. Come up with super bad creative ideas to solve the given Problem. Find 10 possible terrible solutions and list them with name and short descriptions (less then 50 words) 
        
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
    print("\n---\n")  # This adds a separator between the messages for readability
    iteration1_solutions[i] = completion.choices[0].message
    i += 1


#Iteration2 prompt (CTO to get good ideas out)
iteration2_solutions = {}

user_prompt1 = f"""

Imagine you are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Focus on Sense&Act.
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Core business: Sense&Act, PMA, Scada, PLCs.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Find products not already on the market, they have to be new and differenciators
        - Ensure all initiatives align with business profitability.
        - Make investment decisions as if using personal funds.
        - Aim for non-cannibalizing, fringe markets.
        - PLCs are scalable; seek similar scalability in AI applications.
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        - Your work is highly cruical and you are responsible of coming up with the next best innovation.

        Now take a break, consider your rolle and improve all the given possible solutions  from the terrible solutions from the crazy entrepreneur as stated before. Find parts that make sense and give solutions improved based on the initial worst ideas to the given problem.
    """
    

completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0613",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
iteration2_solutions[0] = completion.choices[0].message
   
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

1. The Paper Trail: Implement a system where all production processes are documented on physical paper, requiring employees to physically carry and exchange papers to track progress. This will create a chaotic and inefficient workflow, leading to delays and confusion.

2. Blindfolded Management: Managers will be blindfolded during the production process, relying solely on their intuition to make decisions and provide guidance. This will lead to incorrect assumptions and poor decision-making, resulting in further delays and errors.

3. Randomized Scheduling: Implement a random scheduling system where production tasks and deadlines are assigned randomly each day. This will create chaos and unpredictability, making it impossible to plan and optimize production processes effectively.

4. Invisible Workers: Replace all workers with invisible robots. Without any visual presence, managers will have no visibility or control over the production processes, leading to complete chaos and inefficie